In [1]:
import pandas as pd
import jellyfish
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
raw = pd.read_csv("test.csv", index_col=0)

In [3]:
raw.columns

Index(['business_name', 'business_address', 'business_city', 'business_state',
       'business_postal_code', 'business_latitude', 'business_longitude',
       'business_phone_number', 'inspection_id', 'inspection_date',
       'inspection_score', 'inspection_type', 'violation_id',
       'violation_description', 'risk_category'],
      dtype='object')

In [4]:
raw.tail()

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
50625,iRco Pan Bakery,3717 Mission St,San Francisco,CA,94110,37.737021,-122.424078,NaN,29246_20170621,2017-06-21,87.0,Routine - Unscheduled,29246_20170621_103142,Unclean nonfood contact surfaces,Low Risk
50626,Pan Rico Bakery,3717 Mission St,San Francisco,CA,94110,37.737021,-122.424078,NaN,29246_20151204,2017-12-04,90.0,Routine - Unscheduled,29246_20151204_103142,Unclean nonfood contact surfaces,Low Risk
50627,BURRITO EXPRESS,1601 TARAVAL St,San Francisco,CA,94116,37.742673,-122.483340,NaN,588_20151117,2017-11-17,78.0,Routine - Unscheduled,588_20151117_103116,Inadequate food safety knowledge or lack of ce...,Moderate Risk
50628,BURRITO EXPRESS,1601 TLRAVAA St,San Francisco,CA,94116,37.742673,-122.483340,NaN,588_20151117,2017-11-17,78.0,Routine - Unscheduled,588_20151117_103106,Improper cooking time or temperatures,High Risk
50629,BURRITO EXPRESS,1601 TARAVAL St,San Francisco,CA,94116,37.742673,-122.483340,NaN,588_20170103,2017-01-03,91.0,Routine - Unscheduled,588_20170103_103161,Low risk vermin infestation,Low Risk


In [5]:
raw=raw.dropna(subset=['business_name', 'business_address', 'business_postal_code'])

In [41]:
raw=raw[['business_name', 'business_address', 'business_postal_code']]

In [7]:
# a 10-row mini test df
mni =raw.loc[50610:50619]

In [42]:
mni.shape

(10, 3)

In [9]:
mni

,business_name,business_address,business_postal_code
50610,Miller's East Coast Deli Restaurant,1725 Polk St,94109
50611,Millers East Coast Deli,1725 Polk St,94109
50612,Umamu Birger SF,2184 Union St,94123
50613,Umami Burger SF,2184 tnion SU,94123
50614,Umami Burger SF,8124 Union St,94123
50615,Burger Umami SF,2184 Union St,94123
50616,Jade Garden,3608 BALBOA Street,94121
50617,Jade Garden,3068 BALBOA St,94121
50618,Jade Garden Restaurant,3608 BALBOA St,94121
50619,Jade Garden,0638 BALBOA St,94121


In [172]:
i

{0: [50610, 50611],
 1: [50612, 50613, 50614, 50615],
 2: [50616, 50617, 50618, 50619]}

In [ ]:
idc=clean_dirty_inspection(raw)

In [30]:
def clean_dirty_inspection(df):
    df = df[['business_name', 'business_address', 'business_postal_code']]
    idct = {}
    iterator = 0
    for zipcode, rows in df.groupby("business_postal_code"):
        name_list = list(rows['business_name'])
        addr_list = list(rows['business_address'])
        tuple_list = list(zip(name_list, addr_list))
        for t in tuple_list:
            name, address = t
            rows['name_sim'] = rows.apply(lambda row: fuzz.token_set_ratio(name, row['business_name']), axis = 1)
            rows['addr_sim'] = rows.apply(lambda row: fuzz.token_sort_ratio(address, row['business_address']), axis = 1)
            same = rows.loc[(rows['name_sim'] >= 60) & (rows['addr_sim'] >= 60)]
            same['business_id'] = iterator
            x=(list(same.index))
            if x != []:
                print("finding the unique:", iterator)
                idct[iterator] = x
                iterator += 1
                
            rows = rows[~rows.isin(same).dropna()]
            #df = df[~df.index.isin(same.index)]
    return idct

In [ ]:
i =clean_dirty_inspection(raw[:5000])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


finding the unique: 0
finding the unique: 1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


finding the unique: 2
finding the unique: 3
finding the unique: 4
finding the unique: 5
finding the unique: 6
finding the unique: 7
finding the unique: 8
finding the unique: 9
finding the unique: 10
finding the unique: 11
finding the unique: 12
finding the unique: 13
finding the unique: 14
finding the unique: 15
finding the unique: 16
finding the unique: 17
finding the unique: 18
finding the unique: 19
finding the unique: 20
finding the unique: 21
finding the unique: 22
finding the unique: 23
finding the unique: 24
finding the unique: 25
finding the unique: 26
finding the unique: 27
finding the unique: 28
finding the unique: 29


In [35]:
i

{0: [39, 153, 597, 821, 997],
 1: [57],
 2: [64, 65, 563, 667, 760],
 3: [80, 775],
 4: [88],
 5: [115],
 6: [146, 405, 742, 809, 814, 892],
 7: [147],
 8: [157],
 9: [159, 844, 891],
 10: [165],
 11: [174],
 12: [176],
 13: [187, 283, 336, 386, 444, 845, 1007],
 14: [226],
 15: [237],
 16: [257, 545],
 17: [289, 333, 720, 733, 868],
 18: [320, 328],
 19: [359],
 20: [375],
 21: [416, 417],
 22: [419],
 23: [451, 620],
 24: [482],
 25: [484],
 26: [534],
 27: [557],
 28: [674],
 29: [680, 851],
 30: [731],
 31: [743],
 32: [798],
 33: [897],
 34: [963],
 35: [1010],
 36: [3, 32, 673, 730],
 37: [7, 247, 869],
 38: [18, 529, 707, 721],
 39: [19, 1006],
 40: [38, 695, 696],
 41: [41],
 42: [42, 70, 381, 400, 879, 904],
 43: [55],
 44: [62],
 45: [69, 240],
 46: [75, 759],
 47: [84],
 48: [93],
 49: [112, 870],
 50: [122, 443, 489, 637, 931],
 51: [137, 260, 949],
 52: [139, 805],
 53: [190, 310],
 54: [205],
 55: [222],
 56: [251, 797],
 57: [259, 264, 477, 716],
 58: [277],
 59: [282],


In [36]:
def link_record(df, idct):
    '''
    Using the dicitionary created, matching the unique resturant's business_id to index in both
    rest_df and insp_df
    '''
    insp_columns = ['inspection_id', 'inspection_date', 'inspection_score', 'inspection_type', 'violation_id', 'violation_description', 'risk_category']
    insp_df = df[insp_columns]
    rest_columns = ['business_name', 'business_address', 'business_city','business_state', 'business_postal_code', 'business_latitude', 'business_longitude', 'business_phone_number']                    
    rest_df = df[rest_columns]
    
    
    for k, v in idct.items():
        insp_df.loc[v, "business_id"]= int(k)
        rest_df.loc[v[-1], 'business_id']= int(k)
    rest_df = rest_df.dropna(subset=["business_id"])
    return insp_df, rest_df
    

In [37]:
ins, res =link_record(raw[:1000], i)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [38]:
ins

,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,business_id
0,1757_20170928,2017-09-28,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk,157.0
1,4864_20161206,2017-12-06,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk,210.0
2,79782_20160503,2017-05-03,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk,211.0
3,73840_20171207,2017-12-07,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk,36.0
4,76437_20160329,2017-03-29,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk,133.0
5,82123_20171207,2017-12-07,87.0,Routine - Unscheduled,82123_20171207_103131,Moderate risk vermin infestation,Moderate Risk,134.0
6,1146_20180223,2017-02-23,77.0,Routine - Unscheduled,1146_20180223_103103,High risk food holding temperature,High Risk,158.0
7,3838_20180222,2017-02-22,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk,37.0
8,38929_20160114,2017-01-14,88.0,Routine - Unscheduled,38929_20160114_103139,Improper food storage,Low Risk,212.0
9,81161_20160325,2017-03-25,92.0,Routine - Unscheduled,81161_20160325_103149,Wiping cloths not clean or properly stored or ...,Low Risk,213.0


In [39]:
res

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,business_id
9,Limon Peruvian Rotisserie,1001 South Van Ness Ave,San Francisco,CA,94110,NaN,NaN,1.415555e+10,213.0
10,Domino's #7764,876 Geary St,San Francisco,CA,94109,NaN,NaN,NaN,159.0
13,Tartine Manufactory,595 Alabama St,San Francisco,CA,94110,NaN,NaN,1.415548e+10,214.0
14,Hawaiian Drive Inn,2600 San Bruno Ave,San Francisco,CA,94134,37.728718,-122.404101,NaN,569.0
15,Castro Street Chevron,2399 Market St,San Francisco,CA,94114,37.762857,-122.434755,1.415562e+10,322.0
22,Flour Craft Bakery,244 Clement St,San Francisco,CA,94118,NaN,NaN,1.415539e+10,378.0
25,Boudin Restaurant,160 Jefferson 2nd floor,San Francisco,CA,94133,37.808352,-122.415157,NaN,515.0
26,KENTUCKY FRIED CHICKEN #249,691 EDDY St,San Francisco,CA,94109,37.783053,-122.418988,1.415567e+10,160.0
27,Cafe Reveille,201 Steiner St,San Francisco,CA,94117,NaN,NaN,NaN,364.0
37,Chowders,Pier 39 Space A3,San Francisco,CA,94133,37.808240,-122.410189,NaN,516.0


In [29]:
def get_tuple(a, b):
    '''
    Inputs:
        one row from each of the two parallel dataframe

    Returns: a single tuple contains three categories
    '''
    j = jellyfish.jaro_winkler(a, b)
    name_score = get_jw_category(j)

    return name_score

In [31]:
THRESH1 = 0.8
THRESH2 = 1.0

def get_jw_category(j):
    '''
    Convert a Jaro-Winkler score into a categorical: low, medium. high
    using fixed thresholds.

    Inputs:
        j (double): value between 0 and 1 (inclusive)

    Returns: string
    '''
    if j < THRESH1:
        return "low"
    if j < THRESH2:
        return "medium"
    return "high"